# New Sustainability data format.

In [ ]:
from IPython.display import display
from operator import attrgetter
from bson import ObjectId
import pandas as pd
import mongoengine
import pymongo
from lib import string_to_lower_upper, dbref2id, dereference
client = pymongo.MongoClient()
db_ = client.get_database(name = "goldmarie-optimization-db")

In [ ]:
exchange_ = db_.get_collection("exchange")
esg_data_ = db_.get_collection("e_s_g_data")
pai_data_ = db_.get_collection("pai_data")
published_portfolio_ = db_.get_collection("published_portfolio")
instrument_ = db_.get_collection("instrument")
imug_theme_ = db_.get_collection("i_m_u_g_theme")
new_sus_tmp_imug_theme_ = db_.get_collection("new_sus_tmp_imug_theme")

new_sus_data_ = db_.get_collection("new_sus_data")
new_sus_description_ = db_.get_collection("new_sus_description")
new_sus_filter_ = db_.get_collection("new_sus_filter")

In [ ]:
# technical stuff

# ad "new_code" to impact_themes
l = imug_theme_.aggregate([
    {"$sort": {"number": 1}},
    {
        "$project": {
            "number": 1,
            "name": 1,
        }
    }
])
df = pd.DataFrame(l)
df["code"] = df["name"].map(lambda s: s.replace("&", "and").replace(" ", "_").lower())
impact_theme_codes = df[["_id", "code"]]

new_sus_tmp_imug_theme_.drop()
new_sus_tmp_imug_theme_.insert_many(impact_theme_codes.transpose().to_dict().values())

In [ ]:
# Sustainability indicator descriptions:

class SustainabilityFeatureDescription(mongoengine.Document):
    """Description of a sustainability reporting feature

    E.g.:
    code = "CAS.ALC1.c_2"
    name = "Production of alcoholic beverages"
    unit
    description_en = ""
    description_de = ""
    icon = "cas_alc1_c20.svg"
    color = "#0000FF"

    """
    code = mongoengine.StringField(unique=True)
    name_en = mongoengine.StringField()
    name_de = mongoengine.StringField()
    unit = mongoengine.StringField() # optional. Only applies to leaves. Does not necessarily clarify everything without additional information by description.
    description_en = mongoengine.StringField()
    description_de = mongoengine.StringField()
    icon = mongoengine.StringField()
    color = mongoengine.StringField()
    source_column_names = mongoengine.ListField(mongoengine.StringField())


new_sus_description_.drop()
new_sus_description_.insert_many([
    {
        "code": "ESG",
        "name_en": "ESG scoring",
        "name_de": "ESG Score",
        "source_column_names": ["ESG Assessment Data", "ESG Assessment"],
    },
    {
        "code": "ESG.ESG",
        "name_en": "ESG overall score",
        "source_column_names": ["ESG overall score", "ESG Overall Score (Infokriterium)"]
        # the later column can probably be dropped because it contains duplicate information.
    },
    {"code": "ESG.E", "name_en": "Environmental pillar score"},
    {"code": "ESG.S", "name_en": "Social pillar score"},
    {"code": "ESG.G", "name_en": "Governance pillar score"},
    {"code": "EU_taxonomy.eligible", "name_en": "EU taxonomy eligibility", "color": "#FF0000"},
    {"code": "EU_taxonomy.eligible.indicator", "name_en": "EU taxonomy eligible turnover", "description_en": "EU taxonomy eligibile turnover, as percentage", "color": "#FF0000"},
    {"code": "EU_taxonomy.aligned", "name_en": "EU taxonomy alignment", "color": "#00FF00"},
])

In [ ]:
# Populate the new sustainability collection from previous structure.

# get all isins of currently published portfolios
l_ = published_portfolio_.aggregate([
    *dereference(field = "portfolio", collection = "portfolio"),
    *dereference(field = "portfolio.current_info", collection = "portfolio_current_info"),
    {"$unwind": "$portfolio.current_info.components"},
    {"$project": {"instrument_id": "$portfolio.current_info.components.instrument"}}
])
instrument_ids = list(set(pd.DataFrame(l_)["instrument_id"]))
isins_ = instrument_.aggregate([
    {"$match": {"_id": {"$in": instrument_ids}}},
    {"$project": {"isin": 1}}
])
isins = pd.DataFrame(isins_)["isin"].to_list()
print(len(isins))

In [ ]:
# Create New data structure
new_sus_data_.drop()

# Insert documents with ESG, SDG_involvement, energy transition score and EU Taxonomy data.
sus_data_ = list(esg_data_.aggregate([
    {
        "$match": {
            "isin": {"$in": isins},
            "date": {"$gte": pd.Timestamp("2023-01-01")},
        }
    },
    {
        "$lookup": {
            "from": "pai_data",
            "localField": "isin",
            "foreignField": "isin",
            "as": "PAI",
        }
    },
    {
        "$project": {
            "_id": 1,
            "isin": 1,
            "date": 1,
            "ESG.ESG": "$imug.esg_score",
            "ESG.E": "$imug.environment_score",
            "ESG.S": "$imug.social_score",
            "ESG.G": "$imug.governance_score",
            "EU_taxonomy.eligible.bool": "$tax_eligible.bool",
            "EU_taxonomy.eligible.indicator": "$tax_eligible.turnover",
            "EU_taxonomy.aligned.bool": "$tax_aligned.bool",
            "EU_taxonomy.aligned.indicator": "$tax_aligned.turnover",
            "energy_transition_score": "$imug.energy_transition_score",
            "sdg_involvement": 1,
        }
    },
]))
new_sus_data_.insert_many(sus_data_)

# Updated documents with Impact themes
l = esg_data_.aggregate(
    [
        {
            "$match": {
                "isin": {"$in": isins},
                "date": {"$gte": pd.Timestamp("2023-01-01")},
            }
        },
        {"$unwind": "$imug.sdg_themes.criteria"},
        dbref2id("imug.sdg_themes.criteria.description"),
        *dereference("imug.sdg_themes.criteria.description", "new_sus_tmp_imug_theme"),
        {
            "$project": {
                "imug_rating": "$imug.sdg_themes.criteria.imug_rating",
                "isin": 1,
                "theme": "$imug.sdg_themes.criteria.description.code",
            },
        },
    ]
)
df = pd.DataFrame(l)

assert not any(df.duplicated(subset=["_id", "theme"]))

df[["lower", "upper"]] = df["imug_rating"].apply(string_to_lower_upper)
df = df.drop(columns=["imug_rating"])
for _id, group_df in df.groupby("_id"):
    impact_theme_dict = group_df.drop(columns = ["_id", "isin"]).set_index("theme").transpose().to_dict()
    new_sus_data_.update_one(
        {"_id": _id},
        {"$set": {"IMPACT_THEME": impact_theme_dict}}
    )


In [ ]:
# Update documents with CAS data

sus_data_ = esg_data_.aggregate([
    {
        "$match": {
            "isin": {"$in": isins},
            "date": {"$gte": pd.Timestamp("2023-01-01")},
        }
    },
    {"$unwind": "$imug.controversial_activity_screening.criteria"},
    {"$addFields": {"imug.controversial_activity_screening.criteria.esg_data_id": "$_id"}},
    {"$replaceRoot": {"newRoot": "$imug.controversial_activity_screening.criteria"}},
    {"$unwind": "$criteria"},
    {
        "$project": {
            "esg_data_id": 1,
            "cas_code": "$criteria.code",
            "rating": "$criteria.imug_rating",
        }
    },
    {"$match": {"cas_code": {"$ne": "FRACK"}}},
    #{"$replaceRoot": {"newRoot": "$criteria"}},
])

def _transform_subcode(code: str):
    l1code, l2code = code.split("_", maxsplit=1)
    l2code = "c_"+l2code
    return l1code + "." + l2code

def parse_rating(code, rating):
    # all others are float ratings.
    assert code != "HESC.c_1" #NO = 0; OPEN = 1; YES = 2
    bool_ratings = ["ANIM1.c_4", "GMO1.c_2"]

df = pd.DataFrame(sus_data_)
df["cas_code"] = df["cas_code"].map(_transform_subcode)

for _id, ser in df.groupby("esg_data_id"):
    print(_id)
    break
ser.set_index("cas_code")["rating"]

df.rating.unique()

In [ ]:
df = pd.DataFrame(new_sus_data_.aggregate([]))
df.columns = pd.MultiIndex.from_product([df.columns, [""]])

def transform_dict_to_cols(df, col):
    nlevels = df.columns.nlevels
    new_cols = df[col].apply(lambda dictionary: pd.Series(dictionary))
    new_cols.columns = pd.MultiIndex.from_product([[col], new_cols.columns, *([""] for _ in range(nlevels - 2))])
    df = df.drop(columns = [col], level = 0).join(new_cols)
    return df

#taxonomy_cols = df["EU_taxonomy"].apply(lambda dictionary: pd.Series(dictionary))
#taxonomy_cols.columns = pd.MultiIndex.from_product([["EU_taxonomy"], taxonomy_cols.columns, [""]])
##df["EU_taxonomy"] = taxonomy_cols
#df = df.join(taxonomy_cols)

df = transform_dict_to_cols(df, "ESG")
df = transform_dict_to_cols(df, "EU_taxonomy")
df = transform_dict_to_cols(df, "IMPACT_THEME")
df = df.set_index(["isin", "date"]).drop(columns = ["_id"], level = 0)
df.columns = df.columns.map(lambda col_levels: ".".join([l for l in col_levels if l != ""]))
df

In [ ]:
df_col_details = pd.DataFrame(new_sus_description_.aggregate([
    {
        "$match": {
            "code": {"$in": df.columns.to_list()}
        }
    }
]))
display(df_col_details)
names = df_col_details.set_index("code")["name_en"]
df.columns = df.columns.map(names)
df

## Filters

In [ ]:
filters = {
    "energy_transition_score": {"$gt": 13},
    "ESG.E": {"$gte": 30},
}

df = pd.DataFrame(new_sus_data_.aggregate([
    {
        "$match": {
            "date": {"$lte": pd.Timestamp("2023-10-30")},
            **filters,
        }
    },
    {"$sort": {"date": -1}},
    {
        "$group": {
            "_id": "$isin",
            "doc": {"$first": "$$ROOT"},
        }
    },
    {"$replaceRoot": {"newRoot": "$doc"}},
]))
df.drop(columns = ["_id"]).set_index("isin", verify_integrity=True).head(5)

In [ ]:
# Filter object architecture
new_sus_filter_.drop()
new_sus_filter_.insert_one({
    "IMPACT_THEME.access_to_information.lower": {"comp": "gte", "val": 0},
    "ESG.E": {"comp": "gte", "val": 30},
    "ESG.S": {"comp": "gte", "val": 20},
})

df = pd.DataFrame(new_sus_filter_.aggregate([
    {
        "$project": {
            "_id": 0,
        }
    },
    {
        "$addFields": {
            "arr": {"$objectToArray": "$$ROOT"},
        }
    },
    {"$unwind": "$arr"},
    {"$replaceRoot": {"newRoot": "$arr"}},
    {
        "$lookup": {
            "from": "new_sus_description",
            "localField": "k",
            "foreignField": "code",
            "as": "descr",
        }
    },
    {"$unwind": "$descr"},
    {"$addFields": {
        "comp": "$v.comp", # TODO: how to escape DOLLAR?
        "val": "$v.val",
        "name_en": "$descr.name_en",
    }}
]))[["k", "comp", "val", "name_en"]].set_index("k")
display(df)

filters = {}
for k, ser in df.iterrows():
    filters[k] = {"$" + ser["comp"]: ser["val"]}

filters

In [ ]:
df = pd.DataFrame(new_sus_data_.aggregate([
    {
        "$match": {
            "date": {"$lte": pd.Timestamp("2023-10-30")},
            **filters,
        }
    },
    {"$sort": {"date": -1}},
    {
        "$group": {
            "_id": "$isin",
            "doc": {"$first": "$$ROOT"},
        }
    },
    {"$replaceRoot": {"newRoot": "$doc"}},
]))
df.drop(columns = ["_id"]).set_index("isin", verify_integrity=True).head(5)

## Questions
- translatable fields: name / name_de vs name_en / name_de
- CAS intervals to {"lower": lower, "upper": upper}
- Impact themes